#### Create SparkSession

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
            .appName("test_dataframe") \
            .getOrCreate()

### how to create spark dataframe:
  1. read
  ```python
    df1 = spark.read \
            .format("csv") \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .load("sample.txt)
  ```
  2. sql()
  ```python
    df2 = spark.sql("""
    select count(distinct calltype) as distinct_call_type from fire_service_call_view where calltype is not NULL
    """)
  ```
  3. table(): convert spark table to spark dataframe
  ```python
    df3 = spark.table("spark_db_name.table_name")
  ```
  4. range(): create single column dataframe
  ```python
    df4 = spark.range(5)
  ```
  5. createDataFrame(): converts a list (python list, Spark Row, panda dataframe, RDD) into a spark dataframe
  ```python
    df1 = spark.createDataFrame(data_list)
  ```

In [0]:
df1 = spark.range(5)
df1.printSchema()
df1.show()

In [0]:
from datetime import datetime, date
data_list = [(1, 2., "string1", date(2022,1,1), datetime(2022,1,1,12,0)),
            (2, 3., "string2", date(2022,2,1), datetime(2022,1,2,12,0)),
            (3, 4., "string3", date(2022,3,1), datetime(2022,1,3,12,0))]

In [0]:
df1 = spark.createDataFrame(data_list).toDF("a","b","c","d","e")
df1.show()

In [0]:
df1.printSchema()

In [0]:
schema_1 = ["a","b","c","d","e"] 
schema_2 = "a int, b double, c string, d date, e timestamp"

In [0]:
spark.createDataFrame(data_list, schema=schema_1).printSchema()

In [0]:
spark.createDataFrame(data_list, schema=schema_2).printSchema()

In [0]:
from pyspark.sql import Row

row_list = [Row(a=1, b=2., c="string1", d=date(2022,1,1), e=datetime(2022,1,1,12,0)),
            Row(a=2, b=3., c="string2", d=date(2022,2,1), e=datetime(2022,1,2,12,0)),
            Row(a=3, b=4., c="string3", d=date(2022,3,1), e=datetime(2022,1,3,12,0))]

In [0]:
spark.createDataFrame(row_list).printSchema()

In [0]:
import pandas as pd

pd_df = pd.DataFrame({"a": [1,2,3],
                     "b": [2.,3.,4.],
                     "c": ["string1", "string2", "string3"],
                     "d": [date(2022,1,1),date(2022,2,1),date(2022,3,1)],
                     "e": [datetime(2022,1,1,12,0),datetime(2022,1,2,12,0),datetime(2022,1,3,12,0)]})

In [0]:
spark.createDataFrame(pd_df, schema_2).printSchema()

In [0]:
rdd = spark.sparkContext.parallelize([(1, 2., "string1", date(2022,1,1), datetime(2022,1,1,12,0)),
            (2, 3., "string2", date(2022,2,1), datetime(2022,1,2,12,0)),
            (3, 4., "string3", date(2022,3,1), datetime(2022,1,3,12,0))])

In [0]:
spark.createDataFrame(rdd, schema_1).printSchema()

In [0]:
%fs ls /FileStore/tables


In [0]:
flight_schema_ddl = """FL_DATE date, OP_CARRIER STRING, OP_CARRIER_FL_NUM INT, ORIGIN STRING, 
             ORIGIN_CITY_NAME STRING, DEST STRING, DEST_CITY_NAME STRING, CRS_DEP_TIME INT, DEP_TIME INT, 
             WHEELS_ON INT, TAXI_IN INT, CRS_ARR_TIME INT, ARR_TIME INT, CANCELLED STRING, DISTANCE INT"""

1. Schema Inference is not advisable
1. often date, timestamp, boolean datatype can't store correct value into dataframe. Spark dataframe expect date in yyyy-mm-dd format normally. So if the data not loaded properly , it can store null without generating error. So add **mode** option.
2. **mode**: FAILFAST, DROPMALFORMED, PERMISSIVE
3. if there are 2 date columns, one is dd-mm-yyyy and another one is yyyy-mm-dd, then simply load the data as string datatype.
4. enforce schema explicitely.

In [0]:
flight_time_raw_df = spark.read \
                        .format("json") \
                        .schema(flight_schema_ddl) \
                        .option("mode", "FAILFAST") \
                        .option("dateFormat", "M/d/y") \
                        .load("/FileStore/tables/flight_time.json")

In [0]:
flight_time_raw_df.printSchema()

In [0]:
display(flight_time_raw_df)

In [0]:
from pyspark.sql.types import *

flight_schema_struct = StructType([
    StructField("FL_DATE", DateType()),
    StructField("OP_CARRIER", StringType()),
    StructField("OP_CARRIER_FL_NUM", IntegerType()),
    StructField("ORIGIN", StringType()),
    StructField("ORIGIN_CITY_NAME", StringType()),
    StructField("DEST", StringType()),
    StructField("DEST_CITY_NAME", StringType()),
    StructField("CRS_DEP_TIME", StringType()),
    StructField("DEP_TIME", StringType()),
    StructField("WHEELS_ON", StringType()),
    StructField("TAXI_IN", StringType()),
    StructField("CRS_ARR_TIME", StringType()),
    StructField("ARR_TIME", StringType()),
    StructField("CANCELLED", StringType()),
    StructField("DISTANCE", StringType()),
])

In [0]:
flight_time_raw_df1 = spark.read \
                        .format("json") \
                        .schema(flight_schema_struct) \
                        .option("mode", "FAILFAST") \
                        .option("dateFormat", "M/d/y") \
                        .load("/FileStore/tables/flight_time.json")

In [0]:
display(flight_time_raw_df1)

#### There are 2 approaches to write the dataframe
1. read -> transform -> create view using dataframe -> create spark table -> Load into the table
2. read -> transform -> save as a spark table

In [0]:
from pyspark.sql.functions import to_date, expr

flight_time_df = flight_time_raw_df \
                    .withColumn("FL_DATE", to_date("FL_DATE", "M/d/y")) \
                    .withColumn("CANCELLED", expr("if(CANCELLED==1, true, false)"))

#### convert spark dataframe into a spark table 
1. dataframe write mode can be
    1. append
    2. overwrite
    3. error **check
    4.

In [0]:
%fs rm -r /user/hive/warehouse/flight_time_tbl

In [0]:
flight_time_df.write \
            .format("parquet") \
            .mode("overwrite") \
            .saveAsTable("flight_time_tbl")

In [0]:
%sql
describe extended flight_time_tbl

In [0]:
%fs ls /user/hive/warehouse/flight_time_tbl

#### How to check table metadata

In [0]:
%sql
show tables

In [0]:
spark.catalog.listTables()

#### Managed vs External Tables
1. saveAsTable abd Create table DDl by command both create Managed Table default
2. **for Managed Table**, spark creates table at a pre-defined warehouse location. You can specify the location befor creating the cluster by specifying the "spark.sql.warehouse.dir" config. But you can't change once defined.
3. Spark manages table metadata and table data together.
4. spark deletes the table data and metadata both when you drop the table.


1. **External Table** is a mechanism to share the data across the projects or different storage layers.
2. using location in create table ddl command to create external table.
3. You can't drop the external table (drop only metadata not the actual data), but can read, update or overwrite it like managed table.

In [0]:
%sql

create table ext_flight_time_tbl like flight_time_tbl
location "/user/hive/warehouse/flight_time_tbl"

In [0]:
%sql
describe extended ext_flight_time_tbl

In [0]:
spark.catalog.listTables()

In [0]:
%sql
drop table ext_flight_time_tbl